In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### Load Airbnb listing data

In [97]:
q3_21 = pd.read_csv("../Data_Files/listings _202109.csv")
q4_21 = pd.read_csv("../Data_Files/listings_202112.csv")
q1_22 = pd.read_csv("../Data_Files/listings_202203.csv")
q2_22 = pd.read_csv("../Data_Files/listings _202206.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [98]:
# combine dataframes
airbnb_listings = pd.concat([q3_21, q4_21, q1_22, q2_22])

In [99]:
airbnb_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,340,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,76,1,408,2021-06-29,5.03,1,208,58,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.54,1,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,204,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,490,2021-08-14,3.64,1,243,17,NaN


In [100]:
print("Total number of rows in airbnb listings:", len(airbnb_listings.id.unique()))

print("Average number of reviews:", airbnb_listings.number_of_reviews.mean())

print("Average price:", airbnb_listings.price.mean())

print("Average minimum nights:", airbnb_listings.minimum_nights.mean())

Total number of rows in airbnb listings: 50957
Average number of reviews: 24.32397281700734
Average price: 174.24615118376474
Average minimum nights: 21.11678569764578


In [101]:
# show count of listings in each neighborhood
neighborhood_count = airbnb_listings.groupby("neighbourhood").size().reset_index(name="neighborhood_count").sort_values("neighborhood_count", ascending=False)

neighborhood_count.head()

,neighbourhood,neighborhood_count
12,Bedford-Stuyvesant,10919
238,Williamsburg,9972
109,Harlem,7574
30,Bushwick,6568
110,Hell's Kitchen,6523


In [102]:
neighborhood_topten = neighborhood_count.head(10)

In [140]:
### visualize neighbourhood counts
# set size of visualization
plt.figure(figsize=(20,6))

# plot data
bars = plt.bar(neighborhood_topten.neighbourhood, neighborhood_topten.neighborhood_count)

# show value labels
plt.bar_label(bars)

# create y label
plt.ylabel("count", size=12)

# displaying the title
plt.title("Neighborhood count for NYC Airbnb listings Q3 2021-Q2 2022", size=15)

plt.show()

In [104]:
# show count of listings in each borough
neighbourhood_g_count = airbnb_listings.groupby("neighbourhood_group").size().reset_index(name="neighbourhood_g_count").sort_values("neighbourhood_g_count", ascending=False)

neighbourhood_g_count.head()

,neighbourhood_group,neighbourhood_g_count
2,Manhattan,64964
1,Brooklyn,57429
3,Queens,21737
0,Bronx,4709
4,Staten Island,1402


In [105]:
neighbourhood_g_topten = neighbourhood_g_count.head(10)

In [139]:
### visualize borough counts
# set size of visualization
plt.figure(figsize=(10,6))

# plot data
bars = plt.bar(neighbourhood_g_topten.neighbourhood_group, neighbourhood_g_topten.neighbourhood_g_count)

# show value labels
plt.bar_label(bars)

# create y label
plt.ylabel("count", size=10)

# displaying the title
plt.title("Neighbourhood Group count for NYC Airbnb listings Q3 2021-Q2 2022", size=14)

plt.show()

In [107]:
# show count of listings in types of rooms available
room_type_count = airbnb_listings.groupby("room_type").size().reset_index(name="room_type_count").sort_values("room_type_count", ascending=False)

room_type_count.head()

,room_type,room_type_count
0,Entire home/apt,81527
2,Private room,65614
3,Shared room,2239
1,Hotel room,861


In [108]:
room_type_topfive = room_type_count

In [137]:
### visualize room type counts
# set size of visualization
plt.figure(figsize=(8,6))

# plot data
bars = plt.bar(room_type_topfive.room_type, room_type_topfive.room_type_count)

# show value labels
plt.bar_label(bars)

# create y label
plt.ylabel("count", size=10)

# displaying the title
plt.title("Room type count for NYC Airbnb listings Q3 2021-Q2 2022", size=14)

plt.show()

##### Load Airbnb Calendar data

In [110]:
## these file were too large for GitHub. Data can be downloaded from http://insideairbnb.com/get-the-data/
## file to download: calendar.csv.gz

q2_22_cal = pd.read_csv("../Data_Files/calendar_nyc_q2_2022.csv.gz", compression="gzip", header=0, sep=",")
q1_22_cal = pd.read_csv("../Data_Files/calendar_nyc_q1_2022.csv.gz", compression="gzip", header=0, sep=",")
q4_21_cal = pd.read_csv("../Data_Files/calendar_nyc_q4_2021.csv.gz", compression="gzip", header=0, sep=",")
q3_21_cal = pd.read_csv("../Data_Files/calendar_nyc_q3_2021.csv.gz", compression="gzip", header=0, sep=",")

In [111]:
# combine dataframes
airbnb_cal = pd.concat([q3_21_cal, q4_21_cal, q1_22_cal, q2_22_cal])

In [112]:
print("Total number of rows in airbnb calendar listings:", len(airbnb_cal))

Total number of rows in airbnb calendar listings: 54812865


In [113]:
print("Start date is:", airbnb_cal.date.min())
print("End date is:", airbnb_cal.date.max())

Start date is: 2021-09-01
End date is: 2023-06-03


#### Clean price column and summarize mean

In [114]:
# remove na in price column
airbnb_cal = airbnb_cal[airbnb_cal.price.notna()]

# clean price column
new_price = [ p.replace("$", "").replace(",", "") for p in airbnb_cal.price ]

# apply clean prices
airbnb_cal.price = new_price

# round price to 2 decimals
airbnb_cal.price = round(airbnb_cal.price.astype("float"), 2)

# remove records where price = 0
airbnb_cal = airbnb_cal[airbnb_cal.price > 0]

# cast to date
airbnb_cal['date'] = pd.to_datetime(airbnb_cal['date'])

airbnb_cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,57754,2021-09-02,f,285.0,$285.00,2.0,7.0
1,2595,2021-09-02,f,150.0,$150.00,30.0,1125.0
2,2595,2021-09-03,f,150.0,$150.00,30.0,1125.0
3,2595,2021-09-04,f,150.0,$150.00,30.0,1125.0
4,2595,2021-09-05,f,150.0,$150.00,30.0,1125.0


In [115]:
print("Average price is:", airbnb_cal.price.mean())
print("Minimum price is:", airbnb_cal.price.min())
print("Maximum price is:", airbnb_cal.price.max())

Average price is: 177.54264295780425
Minimum price is: 4.0
Maximum price is: 48800.0


In [116]:
# show price statistics
airbnb_cal.price.describe().apply("{0:.4f}".format)

count    54784802.0000
mean          177.5426
std           336.5264
min             4.0000
25%            70.0000
50%           120.0000
75%           195.0000
max         48800.0000
Name: price, dtype: object

In [117]:
# summarize mean price
airbnb_price_day = airbnb_cal.groupby("date")["price"].mean().reset_index(name="mean_price")

# round mean price to 2 decimals
airbnb_price_day.mean_price = round(airbnb_price_day.mean_price, 2)

In [118]:
airbnb_price_day.head()

,date,mean_price
0,2021-09-01,138.77
1,2021-09-02,177.37
2,2021-09-03,178.99
3,2021-09-04,180.16
4,2021-09-05,175.62


In [119]:
airbnb_price_day.tail()

,date,mean_price
636,2023-05-30,199.74
637,2023-05-31,199.82
638,2023-06-01,202.00
639,2023-06-02,206.48
640,2023-06-03,194.80


In [120]:
# get top ten most expensive days
price_topten = airbnb_price_day.sort_values("mean_price", ascending=False).head(10)

In [121]:
price_topten

,date,mean_price
639,2023-06-02,206.48
633,2023-05-27,205.42
632,2023-05-26,205.20
626,2023-05-20,204.90
625,2023-05-19,204.69
618,2023-05-12,203.46
619,2023-05-13,203.28
612,2023-05-06,203.14
611,2023-05-05,202.91
638,2023-06-01,202.00


#### Visualize mean price by day

In [136]:
# set size of visualization
plt.figure(figsize=(15,6))

# plot data
plt.plot(airbnb_price_day.date, airbnb_price_day.mean_price, label="mean_price")

# display legend
plt.legend(loc="upper left")

# create y label
plt.ylabel("Price ($)", size=12)

# displaying the title
plt.title("Daily mean price for NYC Airbnb listings Q3 2021-Q2 2022", size=15)

plt.show()

#### Calculate rolling average

In [123]:
# function to create rolling average given a window size
def calc_rolling_stats(ser, wd_size=7) : return ser.rolling(wd_size, min_periods=1).mean().to_numpy()

In [124]:
# calculate rolling average on 7 day window
rolling_mean = calc_rolling_stats(airbnb_price_day, wd_size=7)

In [125]:
# see shape of rolling means
rolling_mean.shape

(641, 1)

#### Plot daily mean + rolling average prices

In [134]:
# set size of visualization
fig, ax = plt.subplots(figsize=(20, 10))

# plot daily mean price
ax.plot(airbnb_price_day.date, airbnb_price_day.mean_price, color='red', label="mean_price")

# plot rolling mean
ax.plot(airbnb_price_day.date, pd.Series(rolling_mean.reshape(641,)), color='blue', label="rolling_mean")

# create y label
ax.set_ylabel("Price ($)", size=12)

# display legend
plt.legend(loc="upper left")

# displaying the title
plt.title("Daily mean price & rolling mean for NYC Airbnb listings Q3 2021-Q2 2022", size=15)

# plt.show()

In [127]:
airbnb_price_day

,date,mean_price
0,2021-09-01,138.77
1,2021-09-02,177.37
2,2021-09-03,178.99
3,2021-09-04,180.16
4,2021-09-05,175.62
...,...,...
636,2023-05-30,199.74
637,2023-05-31,199.82
638,2023-06-01,202.00
639,2023-06-02,206.48


#### Combine listing & calendar data for Manhattan neighborhood groups

In [128]:
airbnb_cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,57754,2021-09-02,f,285.0,$285.00,2.0,7.0
1,2595,2021-09-02,f,150.0,$150.00,30.0,1125.0
2,2595,2021-09-03,f,150.0,$150.00,30.0,1125.0
3,2595,2021-09-04,f,150.0,$150.00,30.0,1125.0
4,2595,2021-09-05,f,150.0,$150.00,30.0,1125.0


In [129]:
# filter for relevant columns in airbnb_cal
airbnb_cal = airbnb_cal[["listing_id", "date", "price"]]

In [130]:
# filter for Manhattan listings
manhattan_listings = airbnb_listings[airbnb_listings.neighbourhood_group == "Manhattan"]

# filter for relevant columns in airbnb_listings
manhattan_listings_cut = airbnb_listings[["id", "neighbourhood_group", "neighbourhood"]]

# join calendar data onto Manhattan listings
manhattan_cal = airbnb_cal.merge(manhattan_listings_cut, how="inner", left_on="listing_id", right_on="id")

In [131]:
manhattan_cal.head()

,listing_id,date,price,id,neighbourhood_group,neighbourhood
0,57754,2021-09-02,285.0,57754,Manhattan,Flatiron District
1,57754,2021-09-02,285.0,57754,Manhattan,Flatiron District
2,57754,2021-09-02,285.0,57754,Manhattan,Flatiron District
3,57754,2021-09-02,285.0,57754,Manhattan,Flatiron District
4,57754,2021-09-03,285.0,57754,Manhattan,Flatiron District


In [132]:
print("Average daily price in Manhattan:", manhattan_cal.price.mean())

Average daily price in Manhattan: 176.55595263968075


In [133]:
manhattan_cal.groupby("neighbourhood")["price"].mean().reset_index(name="mean_price").sort_values("mean_price", ascending=False)

,neighbourhood,mean_price
93,Fort Wadsworth,738.219178
121,Jamaica Estates,566.692022
180,Prospect Park,531.045010
220,Tribeca,502.722058
22,Briarwood,423.990124
...,...,...
26,Bronxdale,62.396147
202,Soundview,62.381840
101,Graniteville,62.255804
29,Bull's Head,60.900274
